## Boosting

Boosting is another ensemble techinque like random forests. With random forests, we trained multiple decision trees independently and created independence by randomly sampling samples and features.

Boosting on the other hand trains multiple models sequentially where each each model tries to improve on the areas where the previous models performed poorly. 

### Adaboost

Adaboost works by starting with a single model. From that model we then make predictions on the **training** set. We can then see which training samples this first model got right and which it got wrong. Adaboost then trains the next model, but puts more weight on the training samples that the first model got wrong. This process continues for N number of models where N is a hyper-parameter.

It is important to note that the sequential nature of boosting makes it harder to scale and parallelize relative the random forest models. That being said, work has been done with boosting methods to allow them to be more parallelizable. A very popular library that does this is [XGboost](https://github.com/dmlc/xgboost).

So - at the end of your training, you now have N models where each model is trained to do better on the instances that the previous models did poorly on. You can now combine these models via a weighted voting or average method very similar to random forest except you weight each vote by how accurate the model was overall.

Adaboost also had a hyper-parameter called **learning rate.** This hyper-parameter adjusts the contribution of each classifier. When decreasing the value, each new classifier makes smaller adjustments to the weights of mis-classified samples. Basically, meaning Adaboost is slower to learn per tree. Typically a lower learning rate requires more trees to perform well. This value and the number of trees can be tuned using cross-validation. 

#### Math

So, how exactly do we do this?

**Step 1**: Set all sample weights to 1/m when have m training examples

**Step 2**: Train the first model

**Step 3**: Calculate the weighted error of this model, which is simply the sum of the weights of the misclassified
examples divided by the total weight of all samples. We will call this $r_j$ for the $j$th model. The best is zero and worst is 1.

**Step 4**: Calculate the predictor's weight where being more accurate gets a higher weight:

$$\alpha_{j} = \eta log \frac{1-r_{j}}{r_j}$$

If you are wrong more than right you get a negative weight and if close to random weight close to zero

**Step 5**: Update the weights of your training samples where if you got it right, the weight remains the same. If you got it wrong new weight is:

$$old weight * exp(\alpha_{j})$$

Then normalize all weights to sum to 1 by dividing all weights by sum of weights.

You can see that a good predictor adds extra weight to its mis-classification, putting a strong emphasis on them for the next model. Also, $\eta$ is our learning rate and can decrease the impact of a tree on weight updates by being less than 1.

**Step 6**: Repeat from step 2 with next model and continue repeating until required number of models.

That's it!

Predictions are made by running a new sample through all the trained models, getting the most likely class (for classification) for each one, and then doing a weighted vote where the weight is the value from step 4. For regression just weighted average.

Note: Almost always the model choosen for boosting is a decision tree.

### Gradient Boosting

Gradient boosting is also sequential, but instead of changing weights, it uses the residual errors from the previous model as the targets.

Basically for regression, here are our steps:

1. Fit a decision tree (assuming this is our base model and it is the most common)
2. Calculate the residuals: true training values - predicted training values. Note: it turns out that this is the same as taking the negative gradient of the loss function, so this can generalize to other loss functions for classification and ranking tasks.
3. Train a second decision tree where the residuals are the targets
4. Continue the process for the number of defined estimators (hyper-parameter)

At prediction time, we make a prediction with each tree and **sum** them. 

### Learning Rate

The learning rate for gradient boosting trees, scales the contribution of each tree. 


### Early Stopping

A good way to decide how many trees are needed is to use early stopping. See page 199 of Hands on Machine Learning for an example of this with sklearn. Basically, as you add an additional tree, you check your validation error and when your validation error stops getting better, you stop adding trees.


### Some final notes

* Boosting is more likely to overfit than random forests when the number of estimators is large. Though, usually slow to overfit.
* Typical learning rates are around 0.01 or 0.001. And small learning rates can require a large number of estimators to achieve good results.
* The max depth of the trees controls the complexity of the model and a max depth of 1 can often work well. This results in an additive model.
* Can also get feature importance scores as with Random Forest.

### Stacking

We won't spend time discussing stacking as it tends to be too complex for industry. That being said, it is a good technique to be familiar with. Take a look at p. 200 of Hands on Machine Learning.

## SKlearn Example

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from collections import Counter
import numpy as np
import pandas as pd
%matplotlib inline

In [4]:
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.20, random_state=42)

In [5]:
clf = GradientBoostingClassifier()
gridsearch = GridSearchCV(clf, {"learning_rate": [.1, .01, .001], "n_estimators": [100, 1000, 5000], 
                                'max_depth': [1, 2, 3]}, scoring='f1')
gridsearch.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [100, 1000, 5000], 'max_depth': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [10]:
print("Best Params: {}".format(gridsearch.best_params_))
print("\nClassification Report:")
print(classification_report(y_test, gridsearch.predict(X_test)))

Best Params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 5000}

Classification Report:
             precision    recall  f1-score   support

          0       0.95      0.93      0.94        43
          1       0.96      0.97      0.97        71

avg / total       0.96      0.96      0.96       114

